In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, roc_auc_score, confusion_matrix, f1_score

import optuna

import psycopg2
from config import *

from datetime import timedelta

import utm
import time

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils import class_weight
import optuna

In [2]:
conexion = psycopg2.connect(database=db_database, 
                                user=db_user, 
                                password=db_password, 
                                host=db_host, 
                                port=db_port)

In [3]:
fechaHasta='2023-01-01'
pathModelos = '/data/proyectos/GRAPEVINE/Models/fjlacueva/Phenology/GBX'
query="select * from public.sabana_sin_rad where fecha <= '"+ fechaHasta + "';"
sabana = pd.read_sql_query(query, con=conexion).drop_duplicates()
sabana

,phenologystageid,variedad,codigo,longitude,latitude,altitud,doc_id,fecha,estacion,season,...,do_Borja,do_Calatayud,do_Cariñena,do_Somontano,variedad_CABERNET SAUVIGNON,variedad_CHARDONNAY,variedad_GARNACHA,variedad_MAZUELA,variedad_SYRACH,variedad_TEMPRANILLO
0,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-05,Z01,2015_2016,...,0,0,1,0,0,0,1,0,0,0
1,0.0,TEMPRANILLO,50004A02500051,-1.253609,41.357499,587.0,Cariñena,2016-01-05,Z01,2015_2016,...,0,0,1,0,0,0,0,0,0,1
2,0.0,GARNACHA,50073A03200052,-1.231152,41.350539,578.0,Cariñena,2016-01-05,Z01,2015_2016,...,0,0,1,0,0,0,1,0,0,0
3,0.0,MAZUELA,50201A00200124,-1.225159,41.293432,702.0,Cariñena,2016-01-05,Z01,2015_2016,...,0,0,1,0,0,0,0,1,0,0
4,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-06,Z01,2015_2016,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104630,0.0,GARNACHA,50037A01500059,-1.565603,41.150106,868.0,Calatayud,2022-12-29,Z18,2022_2023,...,0,1,0,0,0,0,1,0,0,0
104631,0.0,GARNACHA,50037A01500059,-1.565603,41.150106,868.0,Calatayud,2022-12-30,Z18,2022_2023,...,0,1,0,0,0,0,1,0,0,0
104632,0.0,GARNACHA,50037A01500059,-1.565603,41.150106,868.0,Calatayud,2022-12-31,Z18,2022_2023,...,0,1,0,0,0,0,1,0,0,0
104633,0.0,GARNACHA,50037A01500059,-1.565603,41.150106,868.0,Calatayud,2023-01-01,Z18,2022_2023,...,0,1,0,0,0,0,1,0,0,0


In [4]:
sabana['doc_id']=sabana.doc_id.fillna('New')

In [5]:
sabana.doc_id.value_counts()

Cariñena     68387
Borja        28918
Calatayud     7141
Somontano      189
Name: doc_id, dtype: int64

In [6]:
parcelas_train=[]
parcelas_test=[]
parcelas_vali=[]
for do in sabana.doc_id.unique():
    print(do)
    parcelas_do_train, parcelas_do_no=train_test_split(list(sabana[sabana.doc_id==do].codigo.unique()),
                                                       random_state=17,test_size=0.4)
    parcelas_train=parcelas_train+parcelas_do_train
    if len(parcelas_do_no)>1:
        parcelas_do_vali, parcelas_do_test=train_test_split(parcelas_do_no,test_size=0.5, random_state=17)
        parcelas_train=parcelas_train+parcelas_do_train
        parcelas_test=parcelas_test+parcelas_do_test
        parcelas_vali=parcelas_vali+parcelas_do_vali
    else:
        pascelas_test=parcelas_test+parcelas_do_no
        

Cariñena
Borja
Calatayud
Somontano


In [7]:
parcelas_vali

['50073A08400031',
 '50073A03500094',
 '50182A01500002',
 '50004A03400175',
 '50073A02000066',
 '50018A03200112',
 '50073A03400128',
 '50073A08900011',
 '50018A00900050',
 '50004A02500059',
 '50295A03900002',
 '50010A00700271',
 '50006A02600039',
 '50114A01800135',
 '50114A02300049',
 '50154A02400063',
 '50154A00500730',
 '50046A01700135',
 '50170A01000234',
 '50029A00800166']

In [8]:
conexion.close()

In [9]:
sabana.shift(-7)

,phenologystageid,variedad,codigo,longitude,latitude,altitud,doc_id,fecha,estacion,season,...,do_Borja,do_Calatayud,do_Cariñena,do_Somontano,variedad_CABERNET SAUVIGNON,variedad_CHARDONNAY,variedad_GARNACHA,variedad_MAZUELA,variedad_SYRACH,variedad_TEMPRANILLO
0,0.0,MAZUELA,50201A00200124,-1.225159,41.293432,702.0,Cariñena,2016-01-06,Z01,2015_2016,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-07,Z01,2015_2016,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,TEMPRANILLO,50004A02500051,-1.253609,41.357499,587.0,Cariñena,2016-01-07,Z01,2015_2016,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,GARNACHA,50073A03200052,-1.231152,41.350539,578.0,Cariñena,2016-01-07,Z01,2015_2016,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,MAZUELA,50201A00200124,-1.225159,41.293432,702.0,Cariñena,2016-01-07,Z01,2015_2016,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
temp=sabana.copy()
datos_list=[]
for id_terr in temp.codigo.unique():
    datos_camp_terr=temp[temp['codigo']==id_terr]
    datos_camp_terr['next_fecha']=datos_camp_terr['fecha'].shift(-7)
    datos_camp_terr['next_pheno']=datos_camp_terr['phenologystageid'].shift(-7)
    datos_camp_terr=datos_camp_terr[(datos_camp_terr.next_fecha-datos_camp_terr.fecha).dt.days==7]
    datos_list.append(datos_camp_terr)
datos_total=pd.concat(datos_list).reset_index(drop=True)
datos_total

/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/glabata/.conda/envs/grapevine/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,phenologystageid,variedad,codigo,longitude,latitude,altitud,doc_id,fecha,estacion,season,...,do_Cariñena,do_Somontano,variedad_CABERNET SAUVIGNON,variedad_CHARDONNAY,variedad_GARNACHA,variedad_MAZUELA,variedad_SYRACH,variedad_TEMPRANILLO,next_fecha,next_pheno
0,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-05,Z01,2015_2016,...,1,0,0,0,1,0,0,0,2016-01-12,0.0
1,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-06,Z01,2015_2016,...,1,0,0,0,1,0,0,0,2016-01-13,0.0
2,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-07,Z01,2015_2016,...,1,0,0,0,1,0,0,0,2016-01-14,0.0
3,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-08,Z01,2015_2016,...,1,0,0,0,1,0,0,0,2016-01-15,0.0
4,0.0,GARNACHA,50004A03200057,-1.284122,41.332883,686.0,Cariñena,2016-01-09,Z01,2015_2016,...,1,0,0,0,1,0,0,0,2016-01-16,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80476,10.0,CHARDONNAY,22196A00100148,-0.101093,42.094222,481.0,Somontano,2020-08-16,gv05,2019_2020,...,0,1,0,1,0,0,0,0,2020-08-23,10.0
80477,10.0,CHARDONNAY,22196A00100148,-0.101093,42.094222,481.0,Somontano,2020-08-17,gv05,2019_2020,...,0,1,0,1,0,0,0,0,2020-08-24,10.0
80478,10.0,CHARDONNAY,22196A00100148,-0.101093,42.094222,481.0,Somontano,2020-08-18,gv05,2019_2020,...,0,1,0,1,0,0,0,0,2020-08-25,10.0
80479,10.0,CHARDONNAY,22196A00100148,-0.101093,42.094222,481.0,Somontano,2020-08-19,gv05,2019_2020,...,0,1,0,1,0,0,0,0,2020-08-26,10.0


In [11]:
def objective(trial,train,vali):
    variedades=[col for col in train.columns if 'variedad_' in col]
    dos=[col for col in train.columns if 'do_' in col]
    variables_basic=['phenologystageid','dia', 'min', 'max', 'mean', 'std', 'median','diff']+variedades+dos
    medidas=variables_basic
    
    altitud=trial.suggest_categorical('Altitud',[True,False])
    latitud=trial.suggest_categorical('Latitud',[True,False])
    longitud=trial.suggest_categorical('Longitud',[True,False])
    if(longitud):
        medidas.append('longitude')
    if(latitud):
        medidas.append('latitude')
    if(altitud):
        medidas.append('altitud')
        
    if (trial.suggest_categorical('Acumuladas',[True,False])):
        
        inicio=trial.suggest_categorical('Inicio mediciones',['t0','1','2'])
        suffix=''
        name_vars=[]
        
        chilling=trial.suggest_categorical('Chilling',[True,False])
        winkler=trial.suggest_categorical('Winkler',[True,False])
        gdd=trial.suggest_categorical('gdd',[True,False])
        
        acumulativo=trial.suggest_categorical('Acumulativo',[True,False])
        if (acumulativo):
            suffix='_Cumm'
            if(trial.suggest_categorical('Precipitaciones',[True,False])):
                name_vars.append('precip__'+inicio+'_')
                
        if (winkler|gdd):
            tbase=trial.suggest_categorical('Tbase',['Tbase','TbaseMax'])
            temperatura_inicio=trial.suggest_categorical('Temperatura inicio',['10.0','4.5'])
        if (chilling):
            tbase_chill=trial.suggest_categorical('Tbase_chilling',['Tbase','Tbasemin','Utah'])
            name_vars.append('chillingDD_7.0_'+inicio+'_' + tbase_chill +'_sum')
        if (winkler):
            if acumulativo:
                name_vars.append('winkler_'+temperatura_inicio+'_'+inicio+'_' + tbase)
            else:
                name_vars.append('winkler_'+temperatura_inicio+'_' + tbase)
        if (gdd):
            name_vars.append('gdd_'+temperatura_inicio+'_'+inicio+'_' + tbase+'_sum')
       
        if len(name_vars)>0:
            semanas_list=[]
            for i in range(1,3):
                if(trial.suggest_categorical(str(i)+'_semanas_atras',[True,False])):
                    semanas_list.append(' '+str(i)+'_semanas_atras')
            if(trial.suggest_categorical('1_semanas_adelante',[True,False])):
                    semanas_list.append(' 1_semanas_adelante')
                    
            for name_var in name_vars:
                medidas.append(name_var+suffix)
                for semana in semanas_list:
                    medidas.append(name_var+suffix+semana)
        
    else:
        name_vars=[]    
        if(trial.suggest_categorical('Temperatura',[True,False])):
            if(trial.suggest_categorical('Temperatura_media',[True,False])):
                name_vars.append('tmed_mean')
            if(trial.suggest_categorical('Temperatura_min',[True,False])):
                name_vars.append('tmed_min')
            if(trial.suggest_categorical('Temperatura_max',[True,False])):
                name_vars.append('tmed_max')

        if(trial.suggest_categorical('Viento',[True,False])):
            if(trial.suggest_categorical('Viento_norte',[True,False])):
                name_vars.append('wind_N')
            if(trial.suggest_categorical('Viento_noreste',[True,False])):
                name_vars.append('wind_NE')
            if(trial.suggest_categorical('Viento_este',[True,False])):
                name_vars.append('wind_E')
            if(trial.suggest_categorical('Viento_sureste',[True,False])):
                name_vars.append('wind_SE')
            if(trial.suggest_categorical('Viento_sur',[True,False])):
                name_vars.append('wind_S')
            if(trial.suggest_categorical('Viento_suroeste',[True,False])):
                name_vars.append('wind_SW')
            if(trial.suggest_categorical('Viento_oeste',[True,False])):
                name_vars.append('wind_W')
            if(trial.suggest_categorical('Viento_noroeste',[True,False])):
                name_vars.append('wind_NW')
        
        if len(name_vars)>0:
            dias_vars=[]
            for i in range(1,14):
                if(trial.suggest_categorical(str(i)+'_dias_atras',[True,False])):
                    dias_vars.append(' ' + str(i) + '_dias_atras')
            for i in range(1,7):
                if(trial.suggest_categorical(str(i)+'_dias_adelante',[True,False])):
                    dias_vars.append(' ' + str(i) + '_dias_adelante')

            for name_var in name_vars:
                medidas.append(name_var)
                for dia_var in dias_vars:
                    medidas.append(name_var+dia_var)

    medidas.append('next_pheno')
    train=train[medidas].dropna()
    vali=vali[medidas].dropna()
    
    print(len(medidas))

    X_train=train.drop(['next_pheno'], axis=1).values
    Y_train=train['next_pheno'].values
    X_vali=vali.drop(['next_pheno'], axis=1).values
    Y_vali=vali['next_pheno'].values

    params = {'random_state':17,'loss':'deviance',
              'n_estimators':int(trial.suggest_discrete_uniform('n_estimators',low=32,high=512,q=32)),
              'validation_fraction':0.3,
              'n_iter_no_change':15,'tol':1e-4,
              'max_depth':trial.suggest_int(f'max_depth',low=3,high=10),
              'min_samples_leaf':float(trial.suggest_discrete_uniform(f'min_samples_leaf',low=0.05,high=0.2,q=0.01)),
              'learning_rate':trial.suggest_discrete_uniform(f'learning_rate',low=5e-3,high=10,q=5e-3)}
    
    model=GradientBoostingClassifier(**params) 
    model.fit(X_train,Y_train)
    preds=model.predict(X_vali)
    
    mcc=matthews_corrcoef(Y_vali,preds)
    print("MCC: ",mcc)
    print("Accuracy: ", accuracy_score(Y_vali,preds))
    print("F1: ", f1_score(Y_vali,preds, average='macro'))
    SCORE=mcc
    
    return mcc

In [ ]:
random_state  = 17
direction     = 'maximize'
n_trials      = 10
n_jobs        = 1
timeout       = None
verbosity     = 0
trozos=500



train=datos_total[datos_total.codigo.isin(parcelas_train)]
vali=datos_total[datos_total.codigo.isin(parcelas_vali)]
train=train.drop(columns=['codigo','fecha'])
print(len(train))
vali=vali.drop(columns=['codigo','fecha'])

study_name = f'Fenologia'
sampler    = optuna.samplers.TPESampler(seed=random_state)

FILE = pathModelos+ f'/resumen_optuna-fenologia-gb-mcc-parcelas.csv'

study = optuna.create_study(study_name=study_name,direction=direction,sampler=sampler,
                            storage='sqlite://///'+ pathModelos + '/optuna-aceituna-fenologia-gb-mcc-parcelas.db', 
                            load_if_exists=True)

for j in range(trozos):
    study.optimize(func=lambda trial: objective(trial,train,vali),n_trials=n_trials,timeout=timeout)

    result_df = study.trials_dataframe()
    result_df.to_csv(FILE)
    time.sleep(1)

[I 2023-10-29 16:38:59,921] A new study created in RDB with name: Fenologia


48799
22


[I 2023-10-29 16:39:13,757] Trial 0 finished with value: 0.39041241066340515 and parameters: {'Altitud': False, 'Latitud': False, 'Longitud': False, 'Acumuladas': True, 'Inicio mediciones': '2', 'Chilling': False, 'Winkler': True, 'gdd': False, 'Acumulativo': True, 'Precipitaciones': False, 'Tbase': 'Tbase', 'Temperatura inicio': '4.5', '1_semanas_atras': True, '2_semanas_atras': True, '1_semanas_adelante': False, 'n_estimators': 352.0, 'max_depth': 4, 'min_samples_leaf': 0.1, 'learning_rate': 9.46}. Best is trial 0 with value: 0.39041241066340515.


MCC:  0.39041241066340515
Accuracy:  0.489540334429441
F1:  0.18259456598163373
44


[I 2023-10-29 16:39:28,240] Trial 1 finished with value: 0.2569162150736176 and parameters: {'Altitud': False, 'Latitud': False, 'Longitud': True, 'Acumuladas': False, 'Temperatura': True, 'Temperatura_media': False, 'Temperatura_min': False, 'Temperatura_max': False, 'Viento': True, 'Viento_norte': False, 'Viento_noreste': True, 'Viento_este': False, 'Viento_sureste': False, 'Viento_sur': False, 'Viento_suroeste': True, 'Viento_oeste': False, 'Viento_noroeste': False, '1_dias_atras': True, '2_dias_atras': True, '3_dias_atras': True, '4_dias_atras': True, '5_dias_atras': True, '6_dias_atras': False, '7_dias_atras': False, '8_dias_atras': True, '9_dias_atras': False, '10_dias_atras': True, '11_dias_atras': False, '12_dias_atras': True, '13_dias_atras': False, '1_dias_adelante': True, '2_dias_adelante': False, '3_dias_adelante': True, '4_dias_adelante': False, '5_dias_adelante': False, '6_dias_adelante': True, 'n_estimators': 160.0, 'max_depth': 10, 'min_samples_leaf': 0.13, 'learning_ra

...

MCC:  0.24267217217659465
Accuracy:  0.33638844189463374
F1:  0.19192550740339076
26


[I 2023-10-29 23:00:03,304] Trial 110 finished with value: 0.6985871021463712 and parameters: {'Altitud': True, 'Latitud': False, 'Longitud': False, 'Acumuladas': True, 'Inicio mediciones': '1', 'Chilling': False, 'Winkler': True, 'gdd': False, 'Acumulativo': True, 'Precipitaciones': True, 'Tbase': 'Tbase', 'Temperatura inicio': '4.5', '1_semanas_atras': True, '2_semanas_atras': True, '1_semanas_adelante': False, 'n_estimators': 384.0, 'max_depth': 8, 'min_samples_leaf': 0.05, 'learning_rate': 0.535}. Best is trial 90 with value: 0.862999083518704.


MCC:  0.6985871021463712
Accuracy:  0.7542153501714126
F1:  0.5120973916178791
26


[I 2023-10-29 23:11:48,087] Trial 111 finished with value: 0.8550979852644041 and parameters: {'Altitud': True, 'Latitud': False, 'Longitud': False, 'Acumuladas': True, 'Inicio mediciones': '1', 'Chilling': False, 'Winkler': True, 'gdd': False, 'Acumulativo': True, 'Precipitaciones': True, 'Tbase': 'Tbase', 'Temperatura inicio': '4.5', '1_semanas_atras': True, '2_semanas_atras': True, '1_semanas_adelante': False, 'n_estimators': 416.0, 'max_depth': 8, 'min_samples_leaf': 0.05, 'learning_rate': 0.14}. Best is trial 90 with value: 0.862999083518704.


MCC:  0.8550979852644041
Accuracy:  0.8828797313370181
F1:  0.6818058321552033
26


In [ ]:
    º